# 2. Statistics

…


## 2.5 TRANSFORMATIONS OF RANDOM VARIABLES 

- We want to go from $x\sim p_x(x)$ to $y\sim p_y(y)$ where $f(x)=y$ is deterministic.


### 2.5.1 Invertible transformations (bijections)
- We can achieve the variable transformation using the determinant of the **Jacobian** of the inverse $f^{-1}$ and $f$ should be a bijection (aka surjective and injective, $f$ has a well defined inverse) (eq 2.257)
- $p_y(y) = p_x(x) \left|\det[J_{f^{-1}}(y)]\right|$

### 2.5.2 Monte Carlo Approximation
- It is often expensive to compute the Jacobian mat so we use Monte Carlo: construct an empirical pdf p(y) based on samples $x^s\sim p(x)$ and compute $y^s\sim f(x^s)$

### 2.5.3 Probability Integral Transform (PIT)
- Basically is a transformation of any pdf to a uniform dist (uses cdf), it is useful because we can define unbiased samples 
- PIT results in Y having a uniform dist given that: $Y(X)=P_X(X)$ is a transformation of the rv $X$, where $X$ has $P_X$ as its cdf.
- The steps are: take samples $x_n\sim p_x$, then compute the empirical cdf $Y=P_X(X)$; then compute the empirical pdf of $Y$ using **kernel density estimates**


## 2.6 MARKOV CHAINS
- The last state $x_t$ captures all relevant info thus it is a sufficient statistic to estimate the future state: $p(x_{x+\tau} | x_t)$, this is called the **Markov Assumption**
- A **Markov Chain** is the joint probability of any sequence of events:
$p(x_{1:T}) = p(x_1) \prod_{t=2}^{T} p(x_t | x_{t-1})$

### 2.6.1 Parametrization
#### 2.6.1.1 Markov transition kernels
- A **transition function / kernel / Markov kernel** is just a conditional dist $p(x_t | x_{t-1})$ that depends on the last state. It satisfies the Kolgomorov axioms
- If we assume that the transition function is *time-independent* then the model is said to be **homogeneous, stationary, or time-invariant**
- The assumption of modeling an arbitrary number of variables w/ fixed number of params is called **parameter typing** (time-invariant is an example of this)

#### 2.6.1.2 Markov transition matrices
- This section deals with discrete vars $X_t \in {1, \ldots, K}$ called **finite-state** Markov chain
- The conditional dist can be written as a $K \times K$ transition matrix $A_{ij} = p(X_t=j | X_{t-1}=i)$ gives the prob of going from state $i$ to state $j$ (each row sums to 1, this called **stochastic matrix**)
- Now we want to get from state $i$ to state $j$ in $n$ steps, what is this prob? Thanks to **Chapman-Kolgomorov** equations this is just powering up a transition matrix $n$ times
    - $A_{ij}(n) = p(X_{t+n}=j | X_t=i)$
    - $A(n) = A^n$
    
#### 2.6.1.3 Higher-order Markov chains
- We can generalize the last case to higher dimensions ie. a model of order w/ memory length $n$, called **Markov model of order n / n-gram model** (technically it should be (n+1)-gram model)
    - $p(x_{1:T}) = p(x_{1:n}) \prod_{t=n+1}^T p( x_t | x_{t-n:t-1})$
    - If $n=1$ this is called a **bigram model**, $n=2$ is trigram, and so on …
- In the following secrions we'll focus on the first-order model. Since we can always convert higher-orders to first-order by defining an augmented state space (containing the $n$ past observations (eq 2.273))
    
### 2.6.2 Application: language modeling
- Markov models have application in LMs, ie. models that can generate (score) a sequence of words 
- conditional language models can be used to generate
sequences given inputs, such as mapping one language to another, or an image to a sequence, etc

### 2.6.3 Parameter Estimation
- Will see how to estimate params of a Markov model

#### 2.6.3.1 Maximum likelihood estimation (MLE)
- The main idea behind MLE is to find the parameter values that maximize the likelihood of observing the given data from a probability function
- Since Markov models have previous states as parameters then its transition matrix contains of the useful info. So MLE is used to estimate the transition states
- Remember that this is basically maximizing the log prob
    - Eg for order 1: $p(x_{1:T}|\boldsymbol{\theta}) = \pi(x_1)A(x_1,x_2)\ldots A(x_{T-1},x_T)$ (notation in Markov models: $\pi(x)=p(x)$)
    - This can be  written w/ prods $\prod$ (eq.2.275) and then generalized to $\mathcal{D}=(\boldsymbol{x_1},\ldots,\boldsymbol{x_N})$ where the chain $x_i$ is $\boldsymbol{x}_i=(x_{i1},\ldots,x_{iN})$ 
    - After taking the log: $\log p(\mathcal{D}|\boldsymbol\theta) = \sum_{i=1}^N\log(p(\boldsymbol{x_i}|\boldsymbol{\theta}))$ and defining counts we arrive to eq.2.276
        - $N_j^1$: represents the number of transition states where state $j$ is the starting state
        - $N_{jk}$: number of transition states from $j$ to $k$ (**bigram statistics**)
        - $N_j$: number of times state $j$ is involved in a state transition (**unigram statistics**)
    - Then we maximize the log likelihood using *Lagrange multipliers* and find the estimates for Markov models: $\hat{\pi}_j=\frac{N_j^1}{\sum_{j'}N_{j'}}$ and $\hat{A}_{jk}=\frac{N_{jk}}{N_j}$
    
#### 2.6.3.2 Sparse data problem
- If we want to fit n-gram models (past state memory) to text data, for large $n$ we face huge overfitting problems fue to sparsity (most words in any training dataset are not linked)
- Thus if we have a chain of length $K$ (# words in dataset), and context $K^{n-1}$ (**context** / past states), then we'll have a sparse matrix of $K\times K$, beacuse most $N_{jk}$ are zero
- Brute force attempts to overcome this have been used in the past [[HNP09](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/35179.pdf)], but ideally we wouldn't like to NOT be able to predict a particular word just because it is not in our training set

#### 2.6.3.3 MAP estimation
- A simple initial solution to the sparsity problem is to use the *Dirichlet prior* $A_{j:}\sim \text{Dir}[\alpha\boldsymbol{1}]$
- The Maximum a Posteriori MAP estimate for transition states is $\hat{A}_{jk}=\frac{N_{jk}+\alpha}{N_j+K\alpha}$ (if $\alpha=1$ is called **add-one-smoothing**)
    - The problem with add-one-smoothing is that it assumes that all n-grams are equally likely (not realistic)
    - More sophisticated approach in Sec.3.7.3, hierarchichal Bayes

### 2.6.4 Stationary distribution of a Markov chain
- Stationary distribution = prob dist that remains unchanged over time

#### 2.6.4.1 What is a stationary distribution?
- The stationary distribution has the interpretation of the limiting distribution when the chain is irreducible and aperiodic
- The formulation is as follows: 
    - the **stationary / equilibrium distribution** case is when (eg in the 1-step transition-mat case) $\boldsymbol{\pi}_t = \boldsymbol{\pi}_{t-1}\boldsymbol{A}$ reaches $\boldsymbol{\pi} = \boldsymbol{\pi}\boldsymbol{A}$ after some iterations, regardless of the initial state dist (eg. prob of being in state $j$ at time $t$ $\pi_t(j)=p(X_t=j)=\sum_{i}\pi_{t-1}A_{ij}$)
    - In general we have the **global balance equations** $\pi_i\sum_{j\neq i}A_{ij} = \sum_{j\neq i}\pi_jA_{ji}$, subject to normalized marginal probs ($=\sum_j\pi_j=1$), so the prob of being in state $i$ times the net flow out of state $i$ must equal the probability of being in each other state $j$ times the net flow from that state into $i$
    
#### 2.6.4.2 Computing the stationary distribution
- The whole thing above is similar to solving an eigenvalue problem $\boldsymbol{A}^T\boldsymbol{v}=\boldsymbol{v}$, where we need to set $\boldsymbol{v}^T=\boldsymbol{\pi}$ 
    - $\boldsymbol{v}$ is an eigenvector w/ eigenval = 1, it exists because $\boldsymbol{A}$ is a row-stochastic-mat ($\boldsymbol{A1}=\boldsymbol{1}$) and eigenvalues for $\boldsymbol{A}, \boldsymbol{A}^T$ are the same
- However we have a few caveats
    - can't handle transition probs equal to 1 or 0, because eigenvectors are only guaranteed to be real-valued if all entries are $A_{ij} > 0$ (and hence $A_{ij} < 1$)
    - a trick is to generalize the eigenvalue problem using the constrains 
        - $K$ constraints in $\boldsymbol{\pi}(\boldsymbol{I}-\boldsymbol{A})=\boldsymbol{0}_{K\times 1}$
        - 1 constraint in $\boldsymbol{\pi}\boldsymbol{1}_{K\times 1}=1$
    - now we need to compute $\boldsymbol{\pi}\boldsymbol{M}=\boldsymbol{r}$ (where $\boldsymbol{M}=[\boldsymbol{I}-\boldsymbol{A}, \boldsymbol{1}]$, $\boldsymbol{r}=[0,\ldots,0,1]$) then drop the last column of $\boldsymbol{I}-\boldsymbol{A}$ and the last zero of $\boldsymbol{r}$ to fix the unscontraint-problem given the size of $M$ being $K\times (K+1)$
- Unfortunately not all chains have stationary dists

#### 2.6.4.3 When does a stationary distribution exists
- Few definition of terms
    - **regularity** a Markov chain that converges to a stationary state. The way to prove this is to find only positive entries in the transition matrix in a finite number of steps
    - **irreducible** property of Markov chains where it is possible to reach any state from any other state in a finite number of steps, ie. all states communicate with each other and ensure convergence to a *unique stationary state*
    - **aperiodic** is a sufficient but not necessary condition to ensure that state $i$ has a self-loop
- *Theorem 2.6.1.* Every irreducible (singly connected), aperiodic finite state Markov chain has a limiting distribution, which is equal to $\pi$, its unique stationary distribution.
- *Theorem 2.6.2.* Every irreducible, **ergodic** Markov chain has a limiting distribution, which is equal to $\pi$, its unique stationary distribution. (generalizes *Theorem 2.6.1.*)

#### 2.6.4.4 Detailed balance
- Because establishing *ergodicity* is difficult we can use an alternative (easier to verify) condition:
    - A **time reversible** Markov chain has **detailed balance equations**: $\pi_iA_{ij}=\pi_jA_{ji}$, so the flow from i to j must equal the flow from j to i, weighted by the appropriate source probabilities
- *Theorem 2.6.3.* If a Markov chain with transition matrix $A$ is regular and satisfies the detailed
balance equations wrt distribution $\pi$, then $\pi$ is a stationary distribution of the chain.

## 2.7 Divergence measures between probability distributions